In [ ]:
from tkinter import filedialog
import pickle
import csv
import tkinter as tk
import json
import re

In [ ]:

#En esta celda permite seleccionar los ficheros csv con ventana y opcionalmente un fichero .pickle donde se guardan los componentes y eventos encontrados anteriormente
#https://stackoverflow.com/a/42398049
# Create Tk root
root = tk.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

csvFiles = filedialog.askopenfilename(multiple=True,filetypes=[("Ficheros CSV",".csv")],title="Selecciona los ficheros CSV")

jsonFile=filedialog.askopenfilename(filetypes=[("Fichero JSON",".json")],title="Selecciona el fichero JSON donde se guarda datos anteriores, si no tienes dale a cancelar")

%gui tk

In [ ]:
#Cargamos los componentes y eventos si se ha seleccionado fichero pickle

if len(jsonFile)==0:
    CM={}
else:
    with open(jsonFile,"r") as file:
        CM=json.load(file)
        print("Cargado el JSON", jsonFile)

In [ ]:
nuevos={} #nuevos componente y evento
for csvFile in csvFiles:
    with open(csvFile,encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=',')
        for row in reader:
            component=row["Component"]
            event=row["Event name"]
            
            descripNotNumber=re.sub("'[^']+'","''",row["Description"]) #eliminamos el texto que esta entre comillas simples
            descripNotNumber=re.sub("\d+", "", descripNotNumber) #eliminamos los numertuos
            
            descriptions=CM.setdefault(component,{}).setdefault(event,[])
            if(descripNotNumber not in descriptions):
                descriptions.append(descripNotNumber)
                nuevos.setdefault(component,{}).setdefault(event,[]).append(descripNotNumber)
                if len(descriptions)>1:
                    print("Componente y evento con varias descripciones diferentes:")
                    print("\tComponent: ",component,"\tEvent:",event,"\tDescriptions: ",descriptions)
            

In [ ]:
#Guardamos como JSON
filename=  jsonFile if len(jsonFile)>0 else "Componentes y eventos.json"
with open(filename, 'w',encoding='utf-8' ) as f:
    json.dump( CM, f,indent=4,sort_keys=True,ensure_ascii=False )
    print("Guardado como JSON")

In [ ]:
if len(jsonFile)>0 and len(nuevos)>0: #guardamos como nuevos elementos si se ha cargado un JSON anterior
    with open("nuevos componentes y eventos.json", 'w',encoding='utf-8' ) as f:
        json.dump( nuevos, f,indent=4,sort_keys=True,ensure_ascii=False )
        print("Guardado como JSON nuevos elementos")